In [6]:
import torch, torch.nn as nn
from torch.autograd import Variable

In [18]:
text = ['BOS', 'How', 'are', 'you', 'EOS']
seq_len = len(text)
batch_size = 1
embedding_size = 1
hidden_size = 1
output_size = 1

In [27]:
random_input = Variable(
    torch.FloatTensor(seq_len, batch_size, embedding_size).normal_(), requires_grad=False)

bi_rnn = torch.nn.RNN(
    input_size=embedding_size, hidden_size=hidden_size, num_layers=1, batch_first=False, bidirectional=True)

bi_output, bi_hidden = bi_rnn(random_input)

# stagger
forward_output, backward_output = bi_output[:-2, :, :hidden_size], bi_output[2:, :, hidden_size:]
staggered_output = torch.cat((forward_output, backward_output), dim=-1)

linear = nn.Linear(hidden_size * 2, output_size)

# only predict on words
labels = random_input[1:-1]

# for language models, use cross-entropy :)
loss = nn.MSELoss()
output = loss(linear(staggered_output), labels)

In [29]:
random_input

tensor([[[-0.8803]],

        [[ 0.2783]],

        [[-0.5334]],

        [[ 0.1402]],

        [[-1.8265]]])

In [30]:
bi_output

tensor([[[-0.5148,  0.9234]],

        [[-0.8300,  0.9204]],

        [[-0.8289,  0.9212]],

        [[-0.8748,  0.9065]],

        [[-0.7102,  0.7827]]], grad_fn=<CatBackward>)

In [31]:
bi_hidden

tensor([[[-0.7102]],

        [[ 0.9234]]], grad_fn=<StackBackward>)

In [23]:
print(forward_output)
print(backward_output)

tensor([[[ 0.1312]],

        [[-0.1392]],

        [[-0.1574]]], grad_fn=<SliceBackward>)
tensor([[[ 0.4242]],

        [[ 0.3554]],

        [[-0.0763]]], grad_fn=<SliceBackward>)


In [24]:
staggered_output

tensor([[[ 0.1312,  0.4242]],

        [[-0.1392,  0.3554]],

        [[-0.1574, -0.0763]]], grad_fn=<CatBackward>)

In [25]:
labels

tensor([[[-1.7170]],

        [[-1.3407]],

        [[-2.5114]]])

In [26]:
output

tensor(3.0677, grad_fn=<MseLossBackward>)